In [1]:
# Every returned Out[] is displayed, not just the last one. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import nltk
import pandas as pd
import numpy as np

This is the comment dataframe. It holds two years worth of comments from NLSS episodes. 

In [3]:
comment_df = pd.read_pickle('Pipeline/comtok_df.pkl')

#comment_df['Tokens'] = comment_df['Comment'].apply(nltk.word_tokenize)

comment_df.head() 
comment_df.tail()
print(len(comment_df), 'comments')

,Date,User,Comment,Tokens
0,"August 24, 2017",Moobot,"Thank you for subbing, IHaveNoPokeClue! :D :D","[Thank, you, for, subbing, ,, IHaveNoPokeClue,..."
1,"August 24, 2017",emperorvespasian,VERY LARGE CONE,"[VERY, LARGE, CONE]"
2,"August 24, 2017",KhanWight,Kreygasm CLIMAX Kreygasm CLIMAX Kreygasm,"[Kreygasm, CLIMAX, Kreygasm, CLIMAX, Kreygasm]"
3,"August 24, 2017",Aheron211,VLC,[VLC]
4,"August 24, 2017",Nightbot,Necropara -> Squeal piggy! Squeal [stop spammi...,"[Necropara, -, >, Squeal, piggy, !, Squeal, [,..."


,Date,User,Comment,Tokens
4923013,"February 29, 2016",BlastrMaster,milwaukee HYPE,"[milwaukee, HYPE]"
4923014,"February 29, 2016",zapranoth,Dan is hosting the nlss while NL is gone,"[Dan, is, hosting, the, nlss, while, NL, is, g..."
4923015,"February 29, 2016",Supermaxman,Sure NL we all know you want the cheese,"[Sure, NL, we, all, know, you, want, the, cheese]"
4923016,"February 29, 2016",Bajur_sauce,have fun in Idaho,"[have, fun, in, Idaho]"
4923017,"February 29, 2016",anonymous_ax,just got here. what'd I miss?,"[just, got, here, ., what, 'd, I, miss, ?]"


4923018 comments


In [4]:
video_df = pd.read_pickle('Pipeline/combined.pkl')
video_df.head()
video_df.tail()
print(len(video_df), 'videos')

,Date,Crew,Docket,RowNum,StartTime,EndTime,ViewsGained,FollowersGained,MaxViewers,AverageViewers,FollowersPerHour,ViewsPerHour,LengthMinutes
0,"August 24, 2017","[Northernlion, RockLeeSmile, CobaltStreak, Alp...","[Passpartout, Party Panic, Pinturillo]",18,2017-08-24 21:15:00.000,2017-08-25 00:15:00.000,5400,91,5751,4715,7.00,415.38,195
1,"August 23, 2017","[Northernlion, RockLeeSmile, LastGreyWolf, HCJ...","[Absolver, Golf It, Quiplash]",19,2017-08-23 21:15:00.000,2017-08-24 00:15:00.000,5962,170,6185,5214,13.08,458.62,195
2,"August 21, 2017","[Northernlion, RockLeeSmile, JSmithOTI, Alpaca...","[Fire Pro Wrestling World, Ultimate Chicken H...",21,2017-08-21 22:15:00.000,2017-08-22 01:15:00.000,5065,95,5347,4797,7.31,389.62,195
3,"August 17, 2017","[RockLeeSmile, LastGreyWolf, HCJustin, BaerTaf...","[Geoguessr, Golf It, Quiplash]",23,2017-08-17 21:15:00.000,2017-08-18 00:15:00.000,5365,108,6075,5214,8.31,412.69,195
4,"August 16, 2017","[Northernlion, BaerTaffy, LastGreyWolf, DanGhe...","[Nidhogg 2, Speedrunners, Pinturillo]",24,2017-08-16 21:15:00.000,2017-08-17 00:30:00.000,5823,116,5250,4681,8.29,415.93,210


,Date,Crew,Docket,RowNum,StartTime,EndTime,ViewsGained,FollowersGained,MaxViewers,AverageViewers,FollowersPerHour,ViewsPerHour,LengthMinutes
194,"August 10, 2015","[Northernlion, RockLeeSmile, JSmithOTI]","[Rebirth, Nuclear Throne, OlliOlli 2: Olliwo...",429,2015-08-10 22:15:00.000,2015-08-11 01:00:00.000,6630,224,7333,6101,18.67,552.50,180
195,"July 30, 2015","[Northernlion, RockLeeSmile]","[Rebirth, Duck Game (continued), Speedrunners]",435,2015-07-30 22:15:00.000,2015-07-31 01:00:00.000,6798,146,5874,4860,12.17,566.50,180
196,"July 30, 2015","[Northernlion, RockLeeSmile]","[Rebirth, Duck Game (continued), Speedrunners]",436,2015-07-30 00:30:00.000,2015-07-30 01:00:00.000,1550,60,4227,3392,12.00,310.00,75
197,"July 29, 2015","[Northernlion, MathasGames, RockLeeSmile, Alpa...","[Rebirth & audio problems, Rocket League, Nu...",437,2015-07-29 22:15:00.000,2015-07-30 00:00:00.000,5050,169,4308,3749,15.36,459.09,165
198,"July 27, 2015","[Northernlion, RockLeeSmile, JSmithOTI]","[Rebirth, Nuclear Throne, Family Feud 2010 (...",438,2015-07-27 22:15:00.000,2015-07-28 01:00:00.000,8887,181,6658,5865,15.08,740.58,180


199 videos


The comments that I currently have available don't go as far back as my video stats do. Lets trim it to fit.

In [5]:
video_df.loc[video_df['Date'] == 'February 29, 2016']

,Date,Crew,Docket,RowNum,StartTime,EndTime,ViewsGained,FollowersGained,MaxViewers,AverageViewers,FollowersPerHour,ViewsPerHour,LengthMinutes
138,"February 29, 2016","[Northernlion, RockLeeSmile, JSmithOTI, Alpaca...","[Afterbirth, Half Dead, Golf With Friends wi...",315,2016-02-29 23:15:00.000,2016-03-01 02:00:00.000,8679,106,7660,7164,8.83,723.25,180


In [6]:
temp_vid = video_df.loc[0:138]
temp_vid.head()
temp_vid.tail()

,Date,Crew,Docket,RowNum,StartTime,EndTime,ViewsGained,FollowersGained,MaxViewers,AverageViewers,FollowersPerHour,ViewsPerHour,LengthMinutes
0,"August 24, 2017","[Northernlion, RockLeeSmile, CobaltStreak, Alp...","[Passpartout, Party Panic, Pinturillo]",18,2017-08-24 21:15:00.000,2017-08-25 00:15:00.000,5400,91,5751,4715,7.00,415.38,195
1,"August 23, 2017","[Northernlion, RockLeeSmile, LastGreyWolf, HCJ...","[Absolver, Golf It, Quiplash]",19,2017-08-23 21:15:00.000,2017-08-24 00:15:00.000,5962,170,6185,5214,13.08,458.62,195
2,"August 21, 2017","[Northernlion, RockLeeSmile, JSmithOTI, Alpaca...","[Fire Pro Wrestling World, Ultimate Chicken H...",21,2017-08-21 22:15:00.000,2017-08-22 01:15:00.000,5065,95,5347,4797,7.31,389.62,195
3,"August 17, 2017","[RockLeeSmile, LastGreyWolf, HCJustin, BaerTaf...","[Geoguessr, Golf It, Quiplash]",23,2017-08-17 21:15:00.000,2017-08-18 00:15:00.000,5365,108,6075,5214,8.31,412.69,195
4,"August 16, 2017","[Northernlion, BaerTaffy, LastGreyWolf, DanGhe...","[Nidhogg 2, Speedrunners, Pinturillo]",24,2017-08-16 21:15:00.000,2017-08-17 00:30:00.000,5823,116,5250,4681,8.29,415.93,210


,Date,Crew,Docket,RowNum,StartTime,EndTime,ViewsGained,FollowersGained,MaxViewers,AverageViewers,FollowersPerHour,ViewsPerHour,LengthMinutes
134,"March 30, 2016","[Northernlion, RockLeeSmile, AlpacaPatrol, Cob...","[Afterbirth, Shellshock Live, Pinturillo 2]",308,2016-03-30 21:15:00.000,2016-03-31 00:00:00.000,9330,135,7226,6149,11.25,777.50,180
135,"March 28, 2016","[Northernlion, AlpacaPatrol, RockLeeSmile, MALF]","[Afterbirth, Space Food Truck, Shellshock Li...",309,2016-03-28 22:15:00.000,2016-03-29 01:15:00.000,10237,130,7401,6301,10.00,787.46,195
136,"March 24, 2016","[Northernlion, RockLeeSmile, LastGreyWolf]","[Afterbirth, The Culling, Golf with Friends]",311,2016-03-24 22:15:00.000,2016-03-25 01:00:00.000,9697,107,6804,5791,8.92,808.08,180
137,"March 23, 2016","[Northernlion, RockLeeSmile, AlpacaPatrol, Cob...","[Afterbirth, Enter the Gungeon, Shellshock L...",312,2016-03-23 21:15:00.000,2016-03-24 00:15:00.000,11778,173,6779,5778,13.31,906.00,195
138,"February 29, 2016","[Northernlion, RockLeeSmile, JSmithOTI, Alpaca...","[Afterbirth, Half Dead, Golf With Friends wi...",315,2016-02-29 23:15:00.000,2016-03-01 02:00:00.000,8679,106,7660,7164,8.83,723.25,180


We need a consistent format for the dates in these two data frames. The two forms currently in use are
- Month DD, YYYY
- YYYY-MM-DD

I find the first one more readable, so let's convert the dates.

months = {'01':'January', '02':'February', '03':'March', '04':'April', '05':'May', '06':'June', '07':'July', '08':'August', '09':'September', '10':'October', '11':'November', '12':'December'}

oldformat = comment_df['Date']
formatted = []

for date in oldformat:
    #Split old format into units
    ymd = date.split('-')
    #Save each piece as a var
    M = ymd[1]
    D = ymd[2]
    Y = ymd[0]
    #Change month numbers to words
    M = months.get(M)
    
    entry = M+' '+D+', '+Y
    
    formatted.append(entry)

Seems to match up now

formatted[0]
formatted[-1]

comment_df['Date'] = formatted
comment_df.head()

comment_df['Tokens'] = comment_df['Comment'].apply(nltk.word_tokenize)

In [10]:
comment_df.head()

,Date,User,Comment,Tokens
0,"August 24, 2017",Moobot,"Thank you for subbing, IHaveNoPokeClue! :D :D","[Thank, you, for, subbing, ,, IHaveNoPokeClue,..."
1,"August 24, 2017",emperorvespasian,VERY LARGE CONE,"[VERY, LARGE, CONE]"
2,"August 24, 2017",KhanWight,Kreygasm CLIMAX Kreygasm CLIMAX Kreygasm,"[Kreygasm, CLIMAX, Kreygasm, CLIMAX, Kreygasm]"
3,"August 24, 2017",Aheron211,VLC,[VLC]
4,"August 24, 2017",Nightbot,Necropara -> Squeal piggy! Squeal [stop spammi...,"[Necropara, -, >, Squeal, piggy, !, Squeal, [,..."


# Analysis

Let's look at the rates of the words 'love' and 'hate' in chat.

In [11]:
import re

In [12]:
#This format oversearches, e.g. 'whatever' contains 'hate'
hate_words = comment_df[comment_df['Comment'].str.contains('hate')]
love_words = comment_df[comment_df['Comment'].str.contains('love')]

In [13]:
len(love_words)
len(hate_words)

13908

8861

In [14]:
with open('PublicData/bad_words.txt') as f:
    bad_words = f.readlines()
    #Remove newline char
    bad_words = [x.strip() for x in bad_words] 
len(bad_words)

451

Is profanity more likely to occur with the words 'love' or 'hate'?

In [15]:
count = 0
for comment in hate_words['Tokens']:
    for word in comment:
        if word in bad_words:
            count+=1
print(count/len(hate_words['Comment'])*100, 'percent of words with \'hate\' contain curse words')

11.082270624083062 percent of words with 'hate' contain curse words


In [16]:
count = 0
for comment in love_words['Tokens']:
    for word in comment:
        if word in bad_words:
            count+=1
print(count/len(hate_words['Comment'])*100, 'percent of words with \'love\' contain curse words')

15.686717074822255 percent of words with 'love' contain curse words


Let's look at the most active users.

In [17]:
comment_df['User'].value_counts()

Nightbot             99774
Dr_Roc               29053
Moobot               27103
ShoresofSerenity     25651
luigimasta777        20301
TheDon_Corleone      18941
SpyroTheSheep        18666
id319                17571
Eluc                 16235
machshop             15431
coosloos             15334
TheobromaCacao       15334
GodlyBurrito         15259
ArrestedHouse        14928
Ptrevordactyl        14169
wolfguy84            14109
juleskum17           13936
Noremac__            13601
charsept             13522
KaiserRolled         12768
Anakhronos           12751
zesty69pepper        12147
graysobstory         12044
Tr11ck               11779
volpescura           11752
Geavil               11325
emperorvespasian     11317
TheAzzaTom           11282
EggianoScumaldoo     11233
ops1197              10651
                     ...  
stepoffbro               1
HerpDerpKnight           1
Bmhicks9911              1
nightjohnayylmao         1
Jib_                     1
lizard01230              1
T

How many comments does a user put out on average?

In [18]:
print(len(comment_df['Comment'])/len(comment_df['User'].unique()), 'comments/user')

83.67498937707147 comments/user


In [19]:
len(comment_df.loc[comment_df['User']=='Sonofabrat'])

127

Some of the top commenters are moderators or tool bots. Ignoring these, what sorts of comments do they produce?

In [20]:
comment_df.loc[comment_df['User']=='Dr_Roc']

,Date,User,Comment,Tokens
102,"August 24, 2017",Dr_Roc,trash panda,"[trash, panda]"
276,"August 24, 2017",Dr_Roc,Put them in plastic bags,"[Put, them, in, plastic, bags]"
310,"August 24, 2017",Dr_Roc,I use them for small trash can bags,"[I, use, them, for, small, trash, can, bags]"
376,"August 24, 2017",Dr_Roc,LUL,[LUL]
456,"August 24, 2017",Dr_Roc,Daddy PP,"[Daddy, PP]"
800,"August 24, 2017",Dr_Roc,"""normal""","[``, normal, '']"
1191,"August 24, 2017",Dr_Roc,morning,[morning]
1236,"August 24, 2017",Dr_Roc,so never?,"[so, never, ?]"
1264,"August 24, 2017",Dr_Roc,rob...,"[rob, ...]"
1304,"August 24, 2017",Dr_Roc,I'm with you NL,"[I, 'm, with, you, NL]"


In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [27]:
foo = []
for post in comment_df.loc[comment_df['User']=='Sonofabrat']['Comment']:
    sentiment = sid.polarity_scores(post)
    nonzero = False
    line = ""
    for k in sorted(sentiment):
        if k == 'compound' and sentiment[k] != 0.0:
            #print('\n'+post)
            nonzero = True
            line+=post+'\n'
        if nonzero:
            line+='{0}: {1}, '.format(k, sentiment[k])+'\n'
            #print('{0}: {1}, '.format(k, sentiment[k]), end='')
    if line != "":
        foo.append(line)

In [28]:
for line in foo[:10]:
    print(line)

The specials at the place I work at are often used to gauge interest in a dish. The best selling ones will sometimes make it to the menu.
compound: 0.802, 
neg: 0.0, 
neu: 0.769, 
pos: 0.231, 

I AGREE
compound: 0.3612, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 

It's worth it since it kickstarted your career
compound: 0.2263, 
neg: 0.0, 
neu: 0.787, 
pos: 0.213, 

I DEMAND TOWERS
compound: -0.128, 
neg: 0.6, 
neu: 0.4, 
pos: 0.0, 

Gunpowder or riot
compound: -0.5574, 
neg: 0.643, 
neu: 0.357, 
pos: 0.0, 

I love the zombie animation
compound: 0.6369, 
neg: 0.0, 
neu: 0.417, 
pos: 0.583, 

FAILURE HAS BEEN SENT
compound: -0.5106, 
neg: 0.524, 
neu: 0.476, 
pos: 0.0, 

Cornfessions of a Cereal Killer
compound: -0.6486, 
neg: 0.589, 
neu: 0.411, 
pos: 0.0, 

:) :) :) :)
compound: 0.9001, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 

Josh looks like a sheriff
compound: 0.3612, 
neg: 0.0, 
neu: 0.545, 
pos: 0.455, 



In [30]:
comment_df.head()

,Date,User,Comment,Tokens
0,"August 24, 2017",Moobot,"Thank you for subbing, IHaveNoPokeClue! :D :D","[Thank, you, for, subbing, ,, IHaveNoPokeClue,..."
1,"August 24, 2017",emperorvespasian,VERY LARGE CONE,"[VERY, LARGE, CONE]"
2,"August 24, 2017",KhanWight,Kreygasm CLIMAX Kreygasm CLIMAX Kreygasm,"[Kreygasm, CLIMAX, Kreygasm, CLIMAX, Kreygasm]"
3,"August 24, 2017",Aheron211,VLC,[VLC]
4,"August 24, 2017",Nightbot,Necropara -> Squeal piggy! Squeal [stop spammi...,"[Necropara, -, >, Squeal, piggy, !, Squeal, [,..."


# Plan

Using machine learning, predict the percentage of positive/negative comments based on features like audience size, show hosts, games played.

Each comment has a compound score that represents an aggregate of its neg, neu, and pos scores. The negative numbers represent overwhelmingly negative comments, and positive numbers represent overwhelmingly positive comments.

As I want to predict the percentage of positive and negative comments overall, I will treat them as two descrete categories based on the compound score. 

In [ ]:
#Holds sentiment values to add as column to df
sent_scores = []

#Iterate through every comment, exluding chat mod bots
for post in comment_df.loc[(comment_df['User']!='Moobot') & (comment_df['User']!='Nightbot')]['Comment']:
    #Use Vader to generate sentiment of the comment
    sentiment = sid.polarity_scores(post)
    nonzero = False

    for k in sorted(sentiment):
        #Ignore sentiment scores where the compound score is 0
        if k == 'compound' and sentiment[k] != 0.0:
            if sentiment[k]>0:
                sent_scores.append('pos')
            if sentiment[k]<0:
                sent_scores.append('neg')
        #If compound is 0 it is unkown, so denote it in the list
        elif k == 'compound' and sentiment[k] == 0.0:
            sent_scores.append('unk')